# Keras custom metric

There are two approaches to implement custom metrics for Keras model training.

1. Metric function

* [Custom metrics for Keras/TensorFlow](https://medium.com/analytics-vidhya/custom-metrics-for-keras-tensorflow-ae7036654e05)


```
def custom_metric_fn(y_true, y_pred): 
    a = tp(y_true, y_pred) 
    b = fp(y_true, y_pred)
    return a - b

model.compile(
    optimizer='adam',
    loss = 'categorical_crossentropy',
    metrics=[custom_metric_fn]
)
```

2. Subclass ```tf.keras.metrics.Metric```
* [tf.keras.metrics.Metric](https://www.tensorflow.org/api_docs/python/tf/keras/metrics/Metric)


> To be implemented by subclasses:
> * ```__init__()```: All state variables should be created in this method by calling self.add_weight() like: self.var = self.add_weight(...)
> * ```update_state()```: Has all updates to the state variables like: self.var.assign_add(...).
> * ```result()```: Computes and returns a scalar value or a dict of scalar values for the metric from the state variables.
> ### Example subclass implementation:  
> ```
> class BinaryTruePositives(tf.keras.metrics.Metric):
>   def __init__(self, name='binary_true_positives', **kwargs):
>     super(BinaryTruePositives, self).__init__(name=name, **kwargs)
>     self.true_positives = self.add_weight(name='tp', initializer='zeros')
> 
>   def update_state(self, y_true, y_pred, sample_weight=None):
>     y_true = tf.cast(y_true, tf.bool)
>     y_pred = tf.cast(y_pred, tf.bool)
> 
>     values = tf.logical_and(tf.equal(y_true, True), tf.equal(y_pred, True))
>     values = tf.cast(values, self.dtype)
>     if sample_weight is not None:
>       sample_weight = tf.cast(sample_weight, self.dtype)
>       sample_weight = tf.broadcast_to(sample_weight, values.shape)
>       values = tf.multiply(values, sample_weight)
>     self.true_positives.assign_add(tf.reduce_sum(values))
> 
>   def result(self):
>     return self.true_positives
> 
> model.compile(loss = loss, optimizer = optimizer, metrics = [BinaryTruePositives()])
> ```